# Simulate Filament Dynamics under Follower-force compression and extension
	- Simulates filament dynamics from different initial orientations.
	- Computes filament orientation before and after one activity cycle (compressive-extensional) cycle

In [31]:
import sys
if 'init_modules' in globals().keys():
    # second or subsequent run: remove all but initially loaded modules
    for m in sys.modules.keys():
        if m not in init_modules:
            del(sys.modules[m])
else:
    # first run: find out which modules were initially loaded
    init_modules = sys.modules.keys()
    
from pyfilaments.activeFilaments import activeFilament
import numpy as np
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
from sys import platform

from joblib import Parallel, delayed
import multiprocessing

# Check which platform
if platform == "linux" or platform == "linux2":
    print("linux system")
    # root_path = '/home/deepak/LacryModelling_Local/SimulationData'
    root_path = '/home/deepak/Dropbox/LacryModeling/ModellingResults'


elif platform == 'darwin':
    print("OSX system")
    root_path = '/Users/deepak/Dropbox/LacryModeling/'

linux system


## Setup filament parameters

In [32]:
radius = 1
bc = {0:'clamped', -1:'free'} # Filament BC

# Bond length
b0 = 2.1

# Activity profile parameters


n_activity_cycles = 1.5 	# No:of activity cycles we want to simulate
activity_timescale = 750	
duty_cycle = 0.5

Tf = n_activity_cycles*activity_timescale
time_step_save = 1
Npts = int(Tf/time_step_save)

init_angle_array = np.linspace(0, np.pi/2, 50)

# bending_stiffness_array = [90, 95, 100]
k = 25

# activity_strength_array = np.linspace(0.5, 3, 20)
activity_strength_array = [1.07692307692308, 1.14102564102564, 2.61538461538461]

## Batch simulation function

In [33]:
save_path = '/home/deepak/ActiveFilamentsSearch_backup_3/FollowerForce_BucklingDynamics_Simulations'

def run_parametric_simulation(pid, init_angle, activity_strength, activity_timescale, Tf):


    fil = activeFilament(dim = 3, Np = 32, radius = radius, b0 = b0*radius, k = k, S0 = 0, D0 = activity_strength, bc = bc)

    print(init_angle)
    fil.simulate(Tf, Npts, save = True, overwrite = False, path = save_path, sim_type = 'point', 
    init_condition = {'shape':'line', 'angle': init_angle}, 
    activity={'type':'square-wave','activity_timescale':activity_timescale, 'duty_cycle':duty_cycle, 'start phase':np.pi},
                note = str(round(init_angle, 2)))


## Run a parallel batch simulation for different parameters and initial angles

In [25]:
num_cores = multiprocessing.cpu_count()

num_cores = 12


for activity_strength in activity_strength_array:

    results = Parallel(n_jobs = num_cores,  verbose=10)(delayed(run_parametric_simulation)(pid, init_angle, activity_strength, activity_timescale, Tf) for pid, init_angle in enumerate(init_angle_array))


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    5.2s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    8.3s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   12.9s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   15.4s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   20.6s remaining:   10.6s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   22.0s remaining:    6.2s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   24.3s remaining:    2.7s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   25.7s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    3.8s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    7.1s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   11.5s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   14.4s
[Parallel(n_jobs=12)]: Done  33 out 

## Batch simulations for different activity time-scales

In [34]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

activity_timescale_array = np.linspace(100, 2000, 5)

activity_strength = 1.65

for activity_timescale in activity_timescale_array:
    Tf = n_activity_cycles*activity_timescale
    time_step_save = 1
    Npts = int(Tf/time_step_save)
    print('Running Activity time-scale: {}...'.format(activity_timescale))
    
    results = Parallel(n_jobs = num_cores,  verbose=10)(delayed(run_parametric_simulation)(pid, init_angle, activity_strength, activity_timescale, Tf) for pid, init_angle in enumerate(init_angle_array))


Running Activity time-scale: 100.0...


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    2.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    2.4s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    3.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    3.5s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:    4.1s remaining:    2.1s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:    4.3s remaining:    1.2s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:    4.6s remaining:    0.5s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:    4.8s finished


Running Activity time-scale: 575.0...


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    3.1s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    5.6s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    9.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   11.1s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   15.1s remaining:    7.8s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   16.4s remaining:    4.6s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   18.2s remaining:    2.0s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   19.3s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


Running Activity time-scale: 1050.0...


[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    6.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    9.9s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   18.2s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   20.5s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   26.4s remaining:   13.6s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   30.1s remaining:    8.5s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   33.8s remaining:    3.8s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   35.9s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


Running Activity time-scale: 1525.0...


[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    9.8s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:   14.7s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   23.6s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   30.9s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   40.0s remaining:   20.6s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   44.8s remaining:   12.6s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   49.1s remaining:    5.5s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   53.2s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


Running Activity time-scale: 2000.0...


[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   10.9s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:   19.1s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   31.5s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   39.9s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   50.1s remaining:   25.8s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   58.2s remaining:   16.4s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:  1.1min remaining:    7.0s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:  1.1min finished
